# Load the Dataset

In [1]:
from datasets import load_dataset
from rich import print
from rich import print

faq_dataset = load_dataset("csv" , data_files="/home/null/code/utd_chatbot/FAQ on UTD Student Accesebility - Sheet1.csv")
print(faq_dataset)
## remove all the None values
faq_dataset = faq_dataset.filter(lambda x: x['Question'] is not None and x['Answering'] is not None)
## reaplace all the word ARC to AccessAbility Resource Center and
'''
Office location: Administration Building, Room 2.224
Email: studentaccess@utdallas.edu (Do not email attachments, upload documents to utd.link/arcupload only.)
Phone: (972) 883-2098
Fax: Please don’t fax, use utd.link/arcupload
Mail: AD 30, 800 West Campbell Rd., Richardson TX 75080
'''
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('ARC', 'AccessAbility Resource Center'), 'Answering': x['Answering'].replace('ARC', 'AccessAbility Resource Center')})

ModuleNotFoundError: No module named 'datasets'

# Preprocess the dataset

## remove all the None values

In [2]:
## remove all the None values
faq_dataset = faq_dataset.filter(lambda x: x['Question'] is not None and x['Answering'] is not None)

Loading cached processed dataset at /home/null/.cache/huggingface/datasets/csv/default-b5279144beba40c5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-f0be635390229960.arrow


## Replace all the ARC to Accessible Resource Center

In [3]:
# replace all the word UTD to University of Texas at Dallas
# replace all the ARC to AccessAbility Resource Center
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('UTD', 'University of Texas at Dallas'), 'Answering': x['Answering'].replace('UTD', 'University of Texas at Dallas')})
faq_dataset = faq_dataset.map(lambda x: {'Question': x['Question'].replace('ARC', 'AccessAbility Resource Center'), 'Answering': x['Answering'].replace('ARC', 'AccessAbility Resource Center')})

Loading cached processed dataset at /home/null/.cache/huggingface/datasets/csv/default-b5279144beba40c5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-9c8a39eaf810fe9d.arrow
Loading cached processed dataset at /home/null/.cache/huggingface/datasets/csv/default-b5279144beba40c5/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-c63ea3f7cd72048e.arrow


# Load the Open AI Emebddigns

In [4]:
import openai
from dotenv import load_dotenv
import os
load_dotenv()  # take environment variables from .env.

OPENKEY_API = os.getenv("OPENAI_KEY")
ORGANIZATION_ID = os.getenv("ORGANIZATION_ID")
openai.organization = ORGANIZATION_ID
# get this from top-right dropdown on OpenAI under organization > settings
openai.api_key = OPENKEY_API
# get API key from top-right dropdown on OpenAI website

openai.Engine.list()  # check we have authenticated
## model of choices
MODEL = "text-similarity-babbage-001"

# Ini the Pinecone Client

In [3]:
import pinecone
PINECONE_APIKEY = os.getenv("PINECONE_APIKEY")
# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key = PINECONE_APIKEY,
    environment="us-east1-gcp"
)

ModuleNotFoundError: No module named 'pinecone'

In [6]:
index = pinecone.Index(index_name="utd-chabot")

# Query a Vector Database

In [7]:
def query_vector_database(query):
    xq = openai.Embedding.create(input=query, engine=MODEL)['data'][0]['embedding']
    res = index.query([xq], top_k=5, include_metadata=True)
    return res

# Role Prompt

In [8]:
role_prompt = """
Please act as a University of Texas at Dallas Counselor. I will provide you with an individual 
looking for guidance at the University of Texas at Dallas, and your task is to help them 
solve their problem\n
"""

# Create an standard QA Propmt

In [9]:
def create_an_standard_qa_prompt( query , res):
    import copy
    role_prompt_temp =  copy.deepcopy(role_prompt)
    for query_response in  res["matches"]:
        try:
            answer = faq_dataset['train'].filter(lambda x: x['Question'] == query_response['metadata']['text'])['Answering'][0]
            role_prompt_temp += f"Q: {query_response['metadata']['text']}\nA: {answer}\n"
        except:
            print(f"The question {query_response['metadata']['text']} is not in the dataset")
            print(f"Errors")
            pass
    prompt = role_prompt_temp + f"Q: {query}\nA:Let’s think step by step."
    return prompt

# Called a LLM Decoder Model

In [10]:
def called_llm_decoder_model(prompt):
  response = openai.Completion.create(
    model="text-davinci-002",
    prompt =  prompt,
    temperature = 0.8,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  return response["choices"][0]["text"]

# Post Process of the Output

In [11]:
def post_process(ans):
    if "ARC" in ans:
        ans = ans.replace("ARC", "AccessAbility Resource Center")
        ans += """ 
        Office location: Administration Building, Room 2.224 \n
        Email: studentaccess@utdallas.edu (Do not email attachments, upload documents to utd.link/arcupload only.) \n
        Phone: (972) 883-2098 \n
        Fax: Please don’t fax, use utd.link/arcupload \n
        Mail: AD 30, 800 West Campbell Rd., Richardson TX 75080    Office location: Administration Building, Room 2.224\n
        """
    return ans

# Gradio

In [12]:
import gradio as gr
questions = []
answers = []
import random
random_value = random.randint(0, 1000)

def utd_chatbot( question ):
    ## store the question in the csvs file
    print(question)
    res = query_vector_database(question)
    print(res)
    propmt = create_an_standard_qa_prompt(question , res)
    print(propmt)
    ans = called_llm_decoder_model(propmt)
    post_process(ans)
    try:
        import pandas as pd
        questions.append( question )
        answers.append( ans )
        data = pd.DataFrame( {"questions": questions, "answers": answers} , columns=["questions", "answers"] )
        data.to_csv(f"user_response_{random_value}.csv")
    except:
        pass
    return ans

In [15]:
## Lauch of Gradio
demo = gr.Interface(fn=utd_chatbot, inputs="text", outputs="text")
demo.launch( share = True )   

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://4296dd86-459a-4b69.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Hi can you help me

{'matches': [{'id': '8',
              'metadata': {'text': 'Services and Procedures for Employees'},
              'score': 0.541972876,
              'sparseValues': {},
              'values': []},
             {'id': '11',
              'metadata': {'text': 'Do I have to submit the form for every '
                                   'test, exam or quiz I want to take with '
                                   'you?\n'},
              'score': 0.586859107,
              'sparseValues': {},
              'values': []},
             {'id': '9',
              'metadata': {'text': 'How do I take a test at the ARC Testing '
                                   'Center?\n'},
              'score': 0.592423201,
              'sparseValues': {},
              'values': []},
             {'id': '7',
              'metadata': {'text': 'What accommodations and services will I be '
                                   'eligible for?'},
              'score': 0.598665833,
              'sparseValues': {},
              'values': []},
             {'id': '13',
              'metadata': {'text': 'I want to change my reservation or the '
                                   'date of the exam or make a change to a '
                                   'reservation I already submitted. Can I do '
                                   'that?'},
              'score': 0.607571,
              'sparseValues': {},
              'values': []}],
 'namespace': ''}

Parameter 'function'=<function create_an_standard_qa_prompt.<locals>.<lambda> at 0x7f6024d43d40> of the transform datasets.arrow_dataset.Dataset.filter@2.0.1 couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
100%|██████████| 1/1 [00:00<00:00, 927.74ba/s]


The question How do I take a test at the ARC Testing Center?
 is not in the dataset

...

100%|██████████| 1/1 [00:00<00:00, 758.88ba/s]


The question Who at the University will know about my disability if I register with ARC? is not in the dataset

Errors

100%|██████████| 1/1 [00:00<00:00, 662.61ba/s]


Please act as a University of Texas at Dallas Counselor. I will provide you with an individual 
looking for guidance at the University of Texas at Dallas, and your task is to help them 
solve their problem

Q: Does UT Dallas provide services for students with disabilities?
A: Yes, accommodations and services are provided for students with documented disabilities to provide equal access 
to educational programs and services in compliance with Section 504 of the Rehabilitation Act of 1973, the 
Americans with Disabilities Act (1990), and UT Dallas policy. 

The AccessAbility Resource Center (AccessAbility Resource Center) facilitates the provision of services and 
accommodations for students. Accommodations are determined on an individual basis using the student’s disability 
documentation and in consultation with the student.
Q: What accommodations and services will I be eligible for?
A: Appropriate and reasonable accommodations are determined on an individual basis. These accommodations may differ
from those provided in high school or other colleges. Accommodations frequently used by students with disabilities 
include extended time for test taking, note takers, printed materials in alternate formats, sign language 
interpreters and the use of assistive technology.
Q: Is there a separate admissions procedure for students with disabilities?

A: No. The admissions process and criteria are the same for all students applying to UT Dallas. Disability status 
will not be a consideration in admissions decisions.


Q: Should I send my disability documentation with my admissions application?
A: No. Documentation should be submitted to AccessAbility Resource Center only after you have been admitted to the 
University and have registered for classes.


Q: I have a disability and I go to UTD - where can I find help?
A:Let’s think step by step.

In [14]:
# Close the a demo
demo.close()

Closing server running on port: 7860
